# Entendiendo modelos
Entender cómo funcionan los modelos entrenando una regresión lineal

Entender:
- Se entrena un modelo por serie o multiple modelos
- Si se obtienen los objetos de los modelos, se pueden utilizar sin tener que utilizar nixtla?
- Los modelos ven las relaciones entre las series

In [1]:
import random
import tempfile
from pathlib import Path

import pandas as pd
from datasetsforecast.m4 import M4
from utilsforecast.plotting import plot_series

from sklearn.linear_model import LinearRegression
from mlforecast import MLForecast

### 0. Data

In [2]:
await M4.async_download('data', group='Hourly')
df, *_ = M4.load('data', 'Hourly')
uids = df['unique_id'].unique()
random.seed(0)
sample_uids = random.choices(uids, k=4)
df = df[df['unique_id'].isin(sample_uids)].reset_index(drop=True)
df['ds'] = df['ds'].astype('int64')

In [3]:
df.head()

,unique_id,ds,y
0,H196,1,11.8
1,H196,2,11.4
2,H196,3,11.1
3,H196,4,10.8
4,H196,5,10.6


### 1. Los modelos ven las relaciones entre las series?
Entrenar el mismo código con un dataframe de una serie y un dataframe de 4 series (la serie de interés y otras 3 más).
Si los resultados son los mismos es porque las ve completamente independiente

#### 1.1 entrenar modelo que ve las 4 series

In [4]:
# obtener dataset con 4 series
df_4series = df.copy()

In [5]:
# crear modelo que ve 4 series
fcst_4series = MLForecast(
    models=[LinearRegression()],
    freq=1,
    lags=[1, 2, 3],
)

In [6]:
fcst_4series.fit(df_4series)

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [7]:
# obtener el ultimo valor real
df_4series.groupby('unique_id').tail(1)

,unique_id,ds,y
1007,H196,1008,16.8
2015,H256,1008,13.4
3023,H381,1008,207.0
4031,H413,1008,34.0


In [8]:
# fcst un horizonte de tiempo
fcst_4series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,18.830526
1,H256,1009,15.671004
2,H381,1009,209.659443
3,H413,1009,32.658524


In [9]:
# obtener la ultima instancia X con los datos transformados. De cada serie
fcst_4series.preprocess(df_4series).groupby('unique_id').tail(1)

,unique_id,ds,y,lag1,lag2,lag3
1007,H196,1008,16.8,17.3,17.8,18.6
2015,H256,1008,13.4,13.8,14.3,14.8
3023,H381,1008,207.0,169.0,148.0,176.0
4031,H413,1008,34.0,41.0,47.0,88.0


#### 1.2 entrenar modelo que ve solo 1 serie

In [10]:
# crear df que tiene solo una serie
df_1series = df[df['unique_id'] == 'H196']
df_1series.head(3)

,unique_id,ds,y
0,H196,1,11.8
1,H196,2,11.4
2,H196,3,11.1


In [11]:
# crear modelo - misma arquitectura que modelo que ve 4 series
fcst_1series = MLForecast(
    models=[LinearRegression()],
    freq=1,
    lags=[1, 2, 3],
)
fcst_1series

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [12]:
fcst_1series.fit(df_1series)

MLForecast(models=[LinearRegression], freq=1, lag_features=['lag1', 'lag2', 'lag3'], date_features=[], num_threads=1)

In [13]:
# obtener el ultimo valor real
df_1series.groupby('unique_id').tail(1)

,unique_id,ds,y
1007,H196,1008,16.8


In [14]:
# fcst un horizonte de tiempo
fcst_1series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,16.36245


In [15]:
# obtener la ultima instancia con los datos transformados
fcst_1series.preprocess(df_1series).groupby('unique_id').tail(1)

,unique_id,ds,y,lag1,lag2,lag3
1007,H196,1008,16.8,17.3,17.8,18.6


#### 1.3 Comparar predicciones para una serie de un modelo que ve una serie vs modelo que ve 4 series
- Si el modelo ve las series de forma independiente, entonces debería exactamente lo mismo.

- Si el modelo considera las relaciones entre las series debería dar distinto. PERO, del notebook siguiente se sabe que el modelo necesita alguna feature input (lag, transformaciones, etc) porque las variables "index, timestamp, y" No se utilizan para entrenar

- Como aquí dan valores distintos las predicciones, entonces, se refuerza la idea de que es un ÚNICO modelo entrenado PASANDO TODAS las series de tiempo SIN IDENTIFICADOR y que el modelo sea lo suficientemente complejo (ensembles) para **identificar patrones en diferentes series de tiempo en los datos que se pasan todas juntas como diferentes observaciones y prediga.** -> Por lo tanto si se pasa 1 serie vs pasar 4 series (la misma serie más otras 3 series) LOS RESULTADOS VAN A SER SIEMPRE DISTINTOS

In [16]:
# ver última observación serie de interés
df_1series.groupby('unique_id').tail(1)

,unique_id,ds,y
1007,H196,1008,16.8


In [17]:
# predicción de modelo que ve solo una serie
fcst_1series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,16.36245


In [18]:
# predicción de modelo que ve 4 series (serie de interés más otras 3)
fcst_4series.predict(1)

,unique_id,ds,LinearRegression
0,H196,1009,18.830526
1,H256,1009,15.671004
2,H381,1009,209.659443
3,H413,1009,32.658524
